In [1]:
from headers import *

In [2]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

In [3]:
df = pd.read_table("RTCI_SP_data.dat")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [4]:
df.drop(df.columns[-1],axis = 1, inplace = True)
database = db.Database("RTCI_SP_surveys",df)

In [5]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4 = Beta('BETA_IVT_4',0,None,None,0)
B_IVT_3 = Beta('BETA_IVT_3',0,None,None,0)
B_IVT_2 = Beta('BETA_IVT_2',0,None,None,0)
B_WAIT_WT = Beta('BETA_WAIT_WT',0,None,None,0)

In [6]:
# variables
NOW_IVT_4 = now_ivt_4
NOW_IVT_3 = now_ivt_3
WAIT_WT_TIME =  wait_WT
WAIT_IVT_3 =  wait_ivt_3
WAIT_IVT_2 = wait_ivt_2

### Pierwszy

In [7]:
# utilities
V1 = ASC_NOW + \
     B_IVT_4 * NOW_IVT_4 + \
     B_IVT_3 * NOW_IVT_3
V2 = ASC_WAIT + \
     B_WAIT_WT * WAIT_WT_TIME + \
     B_IVT_3 * WAIT_IVT_3 + \
     B_IVT_2 * WAIT_IVT_2

### Bez stalej

In [15]:
# utilities
V1 = B_IVT_4 * NOW_IVT_4 + \
     B_IVT_3 * NOW_IVT_3
V2 = B_WAIT_WT * WAIT_WT_TIME + \
     B_IVT_3 * WAIT_IVT_3 + \
     B_IVT_2 * WAIT_IVT_2

In [16]:
# Associate utility functions with the numbering of alternatives
V = {2: V1,
     3: V2}

In [17]:
# Associate the availability conditions with the alternatives

#NOW_AV_SP =  DefineVariable('NOW_AV_SP', 1.0  * (  output_choice   >=  0  ),database)
#WAIT_AV_SP =  DefineVariable('WAIT_AV_SP', 1.0  * (  output_choice   >=  0  ),database)


av = {2: 1,
      3: 1}

In [18]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_EDI_1_no_ASC"

In [19]:
results = biogeme.estimate()

In [20]:
# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_IVT_2=	0.02
BETA_IVT_3=	0.021
BETA_IVT_4=	-0.041
BETA_WAIT_WT=	-0.149
                 Value   Std err     t-test       p-value  Rob. Std err  \
BETA_IVT_2    0.019978  0.002514   7.945984  1.998401e-15      0.002927   
BETA_IVT_3    0.021048  0.001638  12.849319  0.000000e+00      0.001815   
BETA_IVT_4   -0.041026  0.002666 -15.388910  0.000000e+00      0.003231   
BETA_WAIT_WT -0.149018  0.010468 -14.235463  0.000000e+00      0.011189   

              Rob. t-test  Rob. p-value  
BETA_IVT_2       6.824928  8.796963e-12  
BETA_IVT_3      11.595621  0.000000e+00  
BETA_IVT_4     -12.697038  0.000000e+00  
BETA_WAIT_WT   -13.318755  0.000000e+00  


In [ ]:
########################################
#
# @file 01logit.py
# @author: Michel Bierlaire, EPFL
# @date: Thu Sep  6 15:14:39 2018
#
# Logit model
# Three alternatives: Train, Car and Swissmetro
# SP data
#
#######################################

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

# The Pandas data structure is available as database.data. Use all the
# Pandas functions to invesigate the database
#print(database.data.describe())

from headers import *

# Removing some observations can be done directly using pandas.
#remove = (((database.data.PURPOSE != 1) & (database.data.PURPOSE != 3)) | (database.data.CHOICE == 0))
#database.data.drop(database.data[remove].index,inplace=True)

# Here we use the "biogeme" way for backward compatibility
exclude = (( PURPOSE != 1 ) * (  PURPOSE   !=  3  ) +  ( CHOICE == 0 )) > 0
database.remove(exclude)


ASC_CAR = Beta('ASC_CAR',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_SM = Beta('ASC_SM',0,None,None,1)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)

SM_COST =  SM_CO   * (  GA   ==  0  ) 
TRAIN_COST =  TRAIN_CO   * (  GA   ==  0  )

CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)

TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
                                 TRAIN_TT / 100.0,database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
                                   TRAIN_COST / 100,database)
SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
CAR_CO_SCALED = DefineVariable('CAR_CO_SCALED', CAR_CO / 100,database)

V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "01logit"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
